## Decision tree classification on 1994 census to predict if income >50k
* Before we get started with decision trees, we need to convert the categorical variables in our data set to numeric variables.
* The numbers aren't always compatible with other libraries like Scikit-learn, though, so it's easier to just do the conversion to numeric upfront.
* Does this work the same as pd.get_dummies? 

### Notes
* A decision tree is made up of a series of nodes and branches. A node is where we split the data based on a variable, and a branch is one side of the split. 
* When we do our splits, we aren't doing them randomly; we have an objective. Our goal is to ensure that we can make a prediction on future data

### When to stop? aim for the target label only contains one category
* In order to be able to make this prediction, all of the rows in a leaf should only have a single value
* In order to do this, we need a metric for how "together" the different values in the high_income column are.
* The "Entropy", The more "mixed together" 1s and 0s are, the higher the entropy. A data set consisting entirely of 1s in the high_income column would have low entropy
* Information theory: A key concept in information theory is the notion of a bit of information. One bit of information is one unit of information.
* Entropy can be much more complex, especially when we get to cases with more than two possible outcomes, or differential probabilities. 
* We get less than one "bit" of information -- only .97 -- because there are slightly more 1s in the sample data than 0s. This means that if we were predicting a new value, we could guess that the answer is 1 and be right more often than wrong (because there's a .6 probability of the answer being 1). Due to this prior knowledge, we gain less than a full "bit" of information when we observe a new value. >>> this read like Bayes


*bit of information: like the one in Bayes Theorem book, how Turin built the bomb.*

In [ ]:
# Convert a single column from text categories to numbers
col = pandas.Categorical(income["workclass"])
income["workclass"] = col.codes
print(income["workclass"].head(5))
#income["workclass"].unique()

for name in ["education", "marital_status", "occupation", "relationship", "race", "sex", "native_country", "high_income"]:
    col = pandas.Categorical(income[name])
    income[name] = col.codes



In [ ]:
# Seaparate the data into two halves: 
# Enter your code here  > in this case Public or Private sector
private_incomes=income[income['workclass']==4]
public_incomes=income[income['workclass']!=4]


In [ ]:
# Calculate entropy:
import math
# We'll do the same calculation we did above, but in Python
# Passing in 2 as the second parameter to math.log will take a base 2 log
entropy = -(2/5 * math.log(2/5, 2) + 3/5 * math.log(3/5, 2))
print(entropy)

income['high_income'].value_counts()[0]

ones=income['high_income'].value_counts()[1]
zeros=income['high_income'].value_counts()[0]

a=ones/(ones+zeros)
b=zeros/(ones+zeros)


income_entropy = -(a*math.log(a,2)+b*math.log(b,2))

### Which direction to iterate? Use what feature to separate?
* Information gain, or which split can further reduce entropy?
* To compute it, we first calculate the entropy for . Then, for each unique value  in the variable , we compute the number of rows in which  takes on the value , and divide it by the total number of rows. Next, we multiply the results by the entropy of the rows where  is . We add all of these subset entropies together, then subtract from the overall entropy to get information gain.
* To simplify the calculation of information gain and make splits simpler, we won't do it for each unique value. We'll find the median for the variable we're splitting on instead. Any rows where the value of the variable is below the median will go to the left branch, and the rest of the rows will go to the right branch. To compute information gain, we'll only have to compute entropies for two subsets.


In [ ]:
import numpy
def calc_entropy(column):
    """
    Calculate entropy given a pandas series, list, or numpy array.
    """
    # Compute the counts of each unique value in the column
    counts = numpy.bincount(column)
    # Divide by the total column length to get a probability
    probabilities = counts / len(column)
    
    # Initialize the entropy to 0
    entropy = 0
    # Loop through the probabilities, and add each one to the total entropy
    for prob in probabilities:
        if prob > 0:
            entropy += prob * math.log(prob, 2)
    
    return -entropy

# Verify that our function matches our answer from earlier
entropy = calc_entropy([1,1,0,0,1])
print(entropy)

information_gain = entropy - ((.8 * calc_entropy([1,1,0,0])) + (.2 * calc_entropy([1])))
print(information_gain)

# Separate two branches
median_age=income['age'].median()
left_split= income[income['age']<=median_age]
right_split=income[income['age']>median_age]

a=len(left_split)/len(income)
b=len(right_split)/len(income)


age_information_gain= income_entropy - ((a * calc_entropy(left_split['high_income'])) + (b * calc_entropy(right_split['high_income'])))

In [ ]:
# Iterate through each columns to find the one with the most information gain:
def calc_information_gain(data, split_name, target_name):
    """
    Calculate information gain given a data set, column to split on, and target
    """
    # Calculate the original entropy
    original_entropy = calc_entropy(data[target_name])
    
    # Find the median of the column we're splitting
    column = data[split_name]
    median = column.median()
    
    # Make two subsets of the data, based on the median
    left_split = data[column <= median]
    right_split = data[column > median]
    
    # Loop through the splits and calculate the subset entropies
    to_subtract = 0
    for subset in [left_split, right_split]:
        prob = (subset.shape[0] / data.shape[0]) 
        to_subtract += prob * calc_entropy(subset[target_name])
    
    # Return information gain
    return original_entropy - to_subtract

# Verify that our answer is the same as on the last screen
print(calc_information_gain(income, "age", "high_income"))

# My codes
import numpy as np

columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]


information_gains=[]
for col in columns:
    information_gain=calc_information_gain(income, col, "high_income")
    information_gains.append(information_gain)
highest_gain=columns[np.argmax(information_gains)]

---
## Building a decision tree using recursive function
* 

In [ ]:
def find_best_column(data, target_name, columns):
    # Fill in the logic here to automatically find the column in columns to split on
    # data is a dataframe
    # target_name is the name of the target variable
    # columns is a list of potential columns to split on
    information_gains=[]
    for col in columns:
        information_gain=calc_information_gain(data, col, target_name)
        information_gains.append(information_gain)
    best=columns[information_gains.index(max(information_gains))]   
    return best

# A list of columns to potentially split income with
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

income_split=find_best_column(income,'high_income',columns)

In [ ]:
# We'll use lists to store our labels for nodes (when we find them)
# Lists can be accessed inside our recursive function, whereas integers can't.  
# Look at the python missions on scoping for more information on this topic
label_1s = []
label_0s = []

def id3(data, target, columns):
    # The pandas.unique method will return a list of all the unique values in a series
    unique_targets = pandas.unique(data[target])
    
    if len(unique_targets) == 1:
        # Insert code here to append 1 to label_1s or 0 to label_0s, based on what we should label the node
        # See lines 2 and 3 in the algorithm
        if unique_targets==1:
            label_1s.append(1)
        else:
            label_0s.append(0)
        # Returning here is critical -- if we don't, the recursive tree will never finish, and run forever
        # See our example above for when we returned
        return
    
    # Find the best column to split on in our data
    best_column = find_best_column(data, target, columns)
    # Find the median of the column
    column_median = data[best_column].median()
    
    # Create the two splits
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    
    # Loop through the splits and call id3 recursively
    for split in [left_split, right_split]:
        # Call id3 recursively to process each branch
        id3(split, target, columns)
    
# Create the data set that we used in the example on the last screen
data = pandas.DataFrame([
    [0,20,0],
    [0,60,2],
    [0,40,1],
    [1,25,1],
    [1,35,2],
    [1,55,1]
    ])
# Assign column names to the data
data.columns = ["high_income", "age", "marital_status"]

# Call the function on our data to set the counters properly
id3(data, "high_income", ["age", "marital_status"])

In [ ]:
## Use nested disctionary to store nodes info and label
# Create a dictionary to hold the tree  
# It has to be outside of the function so we can access it later
tree = {}

# This list will let us number the nodes  
# It has to be a list so we can access it inside the function
nodes = []

def id3(data, target, columns, tree):
    unique_targets = pandas.unique(data[target])
    
    # Assign the number key to the node dictionary
    nodes.append(len(nodes) + 1) #start from 1, the root
    tree["number"] = nodes[-1]   #the # of that node

    if len(unique_targets) == 1:
        # Insert code here that assigns the "label" field to the node dictionary
        if unique_targets==1:
            tree['label']=1
        else:
            tree['label']=0
        return
    
    best_column = find_best_column(data, target, columns)
    column_median = data[best_column].median()
    
    # Insert code here that assigns the "column" and "median" fields to the node dictionary
    tree['column']=best_column
    tree['median']=column_median
    
    
    left_split = data[data[best_column] <= column_median]
    right_split = data[data[best_column] > column_median]
    split_dict = [["left", left_split], ["right", right_split]]
    
    for name, split in split_dict:
        tree[name] = {}
        id3(split, target, columns, tree[name])

# Call the function on our data to set the counters properly
id3(data, "high_income", ["age", "marital_status"], tree)

In [ ]:
# Print out nested dictionary and its labels:
def print_with_depth(string, depth):
    # Add space before a string
    prefix = "    " * depth
    # Print a string, and indent it appropriately
    print("{0}{1}".format(prefix, string))
    
    
def print_node(tree, depth):
    # Check for the presence of "label" in the tree
    if "label" in tree:
        # If found, then this is a leaf, so print it and return
        print_with_depth("Leaf: Label {0}".format(tree["label"]), depth)
        # This is critical -- without it, you'll get infinite recursion
        return
    # Print information about what the node is splitting on
    print_with_depth("{0} > {1}".format(tree["column"], tree["median"]), depth)
    
    # Create a list of tree branches
    branches = [tree["left"], tree["right"]]
        
    # Insert code here to recursively call print_node on each branch
    for node in branches:
        print_node(node,depth)
    # Don't forget to increment depth when you pass it in
    depth+=1
    
print_node(tree, 0)

In [ ]:
def predict(tree, row):
    if "label" in tree:
        return tree["label"]
    
    column = tree["column"]
    median = tree["median"]
    
    if row[column]<=median:
        return predict(tree['left'],row)
    else:
        return predict(tree['right'],row)
    
    # Insert code here to check whether row[column] is less than or equal to median
    # If it's less than or equal, return the result of predicting on the left branch of the tree
    # If it's greater, return the result of predicting on the right branch of the tree
    # Remember to use the return statement to return the result!

# Print the prediction for the first row in our data
print(predict(tree, data.iloc[0]))

In [ ]:
new_data = pandas.DataFrame([
    [40,0],
    [20,2],
    [80,1],
    [15,1],
    [27,2],
    [38,1]
    ])
# Assign column names to the data
new_data.columns = ["age", "marital_status"]

def batch_predict(tree, df):
    # Insert your code here
    return df.apply(lambda x: predict(tree, x),axis=1)

predictions = batch_predict(tree, new_data)

### Use Scikit-learn to do it!
* AUC ranges from 0 to 1, so it's ideal for binary classification. The higher the AUC, the more accurate our predictions.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# A list of columns to train with
# We've already converted all columns to numeric
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

# Instantiate the classifier
# Set random_state to 1 to make sure the results are consistent
clf = DecisionTreeClassifier(random_state=1)

# We've already loaded the variable "income," which contains all of the income data
clf.fit(income[columns],income['high_income'])

In [ ]:
import numpy
import math

# Set a random seed so the shuffle is the same every time
numpy.random.seed(1)

# Shuffle the rows  
# This permutes the index randomly using numpy.random.permutation
# Then, it reindexes the dataframe with the result
# The net effect is to put the rows into random order
income = income.reindex(numpy.random.permutation(income.index))

train_max_row = math.floor(income.shape[0] * .8)

train=income.iloc[:train_max_row]
test=income.iloc[train_max_row:]

In [ ]:
# Use AUC as the metric to evaluate the performance
from sklearn.metrics import roc_auc_score

clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
error=roc_auc_score(test['high_income'],predictions)
print(error)

In [ ]:
# Improve the model by tweaking parameters to avoid overfitting
clf = DecisionTreeClassifier(random_state=1,min_samples_split=13)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
test_auc=roc_auc_score(test['high_income'],predictions)
print(test_auc)

predictions = clf.predict(train[columns])
train_auc=roc_auc_score(train['high_income'],predictions)
print(train_auc)



In [ ]:
# Experiment on different parameters
clf = DecisionTreeClassifier(random_state=1,min_samples_split=13, max_depth=7)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
test_auc=roc_auc_score(test['high_income'],predictions)
print(test_auc)

predictions = clf.predict(train[columns])
train_auc=roc_auc_score(train['high_income'],predictions)
print(train_auc)



In [ ]:
# Add random noise to test high-variance: difference 
# between test and train will increase because of overfitting


### Notes:
* We can represent the root node with a dictionary, and branches with the keys left and right.
* scikit-learn: sklearn.tree >> DecisionTreeClassifier and DecisionTreeRegressor
* for roc_auc_score > the true/predictions should be in order
* Splitting the data into training and testing sets doesn't prevent overfitting -- it just helps us detect and fix it.
* This may seem to be a strange principle at first, but the deeper a tree is, the worse it typically performs on new data.

#### Three ways to avoid overfitting
* Prune the tree after building one
* Use ensembling to blend the predictions of many trees
* Restrict the depth of the tree

#### Verdict
* By artificially restricting the depth of our tree, we prevent it from creating a model that's complex enough to correctly categorize some of the rows. If we don't perform the artificial restrictions, however, the tree becomes too complex, fits quirks in the data that only exist in the training set, and doesn't generalize to new data.

#### Advantage of Decision Tree
* Easy to interpret
* Relatively fast to fit and make predictions
* Able to handle multiple types of data
* Able to pick up nonlinearities in data, and usually fairly accurate

### Disadvantage:
* Tendency to overfit >>> solution: random forest



---
## Random Forest
* A random forest is an ensemble of decision trees
* fit the data with models with different parameters, features, then come up with a final predictions
* Could be based on majority vote, or simply rounded mean.
* use predict_proba instead (and take the mean across models)

#### Two main ways
* bagging (Bootstrap Aggregation): Some rows from the original data may appear in the "bag" multiple times.
* random feature subsets. 

In [ ]:
# Two simple models, and their auc scores respectively

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score

columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2)
clf.fit(train[columns], train["high_income"])

clf2 = DecisionTreeClassifier(random_state=1, max_depth=5)
clf2.fit(train[columns], train["high_income"])

predictions1=clf.predict(test[columns])
predictions2=clf2.predict(test[columns])

auc1=roc_auc_score(test['high_income'],predictions1)
auc2=roc_auc_score(test['high_income'],predictions2)

print("auc1: ",auc1, "auc2:", auc2)

In [ ]:
# Take the mean of proba across two models
predictions = clf.predict_proba(test[columns])[:,1]
predictions2 = clf2.predict_proba(test[columns])[:,1]

final=numpy.round((predictions+predictions2)/2)

print(roc_auc_score(test['high_income'],final))

In [ ]:
# We'll build 10 trees
tree_count = 10

# Each "bag" will have 60% of the number of original rows
bag_proportion = .6

predictions = []
for i in range(tree_count):
    # We select 60% of the rows from train, sampling with replacement  
    # We set a random state to ensure we'll be able to replicate our results
    # We set it to i instead of a fixed value so we don't get the same sample in every loop
    # That would make all of our trees the same
    bag = train.sample(frac=bag_proportion, replace=True, random_state=i)
    
    # Fit a decision tree model to the "bag"
    clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2)
    clf.fit(bag[columns], bag["high_income"])
    
    # Using the model, make predictions on the test data
    predictions.append(clf.predict_proba(test[columns])[:,1])

final=numpy.round(numpy.mean(predictions,axis=0))

print(roc_auc_score(test['high_income'],final))

In [ ]:
## Use Scikit-learn
# We'll build 10 trees
tree_count = 10

# Each "bag" will have 60% of the number of original rows
bag_proportion = .6

predictions = []
for i in range(tree_count):
    # We select 60% of the rows from train, sampling with replacement
    # We set a random state to ensure we'll be able to replicate our results
    # We set it to i instead of a fixed value so we don't get the same sample every time
    bag = train.sample(frac=bag_proportion, replace=True, random_state=i)
    
    # Fit a decision tree model to the "bag"
    clf = DecisionTreeClassifier(random_state=1, min_samples_leaf=2,splitter='random',max_features='auto')
    clf.fit(bag[columns], bag["high_income"])
    
    # Using the model, make predictions on the test data
    predictions.append(clf.predict_proba(test[columns])[:,1])

combined = numpy.sum(predictions, axis=0) / 10
rounded = numpy.round(combined)

print(roc_auc_score(test["high_income"], rounded))

In [ ]:
## Use RandomForestClassifer
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=5, random_state=1, min_samples_leaf=2)

clf.fit(train[columns],train['high_income'])
predictions=clf.predict(test[columns])
roc_auc_score(test['high_income'],predictions)

### Notes:
* The more "diverse" or dissimilar the models we use to construct an ensemble are, the stronger their combined predictions will be (assuming that all of the models have about the same accuracy).
* Weighted ensemble

#### Scikitlearn
There are also parameters specific to the random forest that alter its overall construction:

n_estimators
bootstrap - "Bootstrap aggregation" is another name for bagging; this parameter indicates whether to turn it on (Defaults to True)


#### Advantage:
* Very accurate predictions - Random forests achieve near state-of-the-art performance on many machine learning tasks. Along with neural networks and gradient-boosted trees, they're typically one of the top-performing algorithms.
* Resistance to overfitting - Due to their construction, random forests are fairly resistant to overfitting. We still need to set and tweak parameters like max_depth though.

#### Weakness:
* They're difficult to interpret - Because we've averaging the results of many trees, it can be hard to figure out why a random forest is making predictions the way it is.
* They take longer to create - Making two trees takes twice as long as making one, making three takes three times as long, and so on. Fortunately, we can exploit multicore processors to parallelize tree construction. Scikit allows us to do this through the n_jobs parameter on RandomForestClassifier. We'll discuss parallelization in greater detail later on.